In [8]:
import multiprocessing
from joblib import Parallel, delayed
import os 
import nibabel as nib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from ipynb.fs.defs.FracTool_Final_Function import FracTool
from ipynb.fs.defs.forfun import FracTool



#load sample slice and convert to np array
example_slice = 'filtered_func_data_clean06.nii.gz'
slice_img = nib.load(example_slice, mmap=False)
slice_array = slice_img.get_fdata()
#pick your slice here (ex: [:,:,20,:]) is slice 20
#pick the range of your signal here (ex: [:,:,20,0:500]) is the first 500 time-points of slice 20)
#slice_sq = slice_array[:,:,20,0:100]
n1_header = slice_img.header
#get TR from header
TR = n1_header.get_zooms()
TR = np.asarray(TR)
TR = TR[3]
[N1, N2, slice_num, N3] = slice_array.shape
print(slice_num)

#images of slice:
#imgplot = plt.imshow(slice_sq[:,:,1])
#imgplot = plt.imshow(slice_sq[:,:,1],cmap='Greys',  interpolation='nearest')
#plt.show()


#[N1,N2,N3] = slice_sq.shape
row = np.arange(0,N1)
column = np.arange(0,N2)
#TR = 2.0 #temporal resolution of signal 

def FracTool_voxel(i, j):
    '''This function returns the Hurst coefficient and class of each voxel in the slice'''
    global TR
    rawbold = (slice_sq[i,j])
    result = FracTool(rawbold, TR) #run Fractool on each signal in signal array
    H_val = result[1] #result[1] of FracTool is Hurst value
    Class_val = result[0] #result[0] of FracTool is Class
    return result #result is array of 2 elements

#multiprocessing for loop
num_cores = multiprocessing.cpu_count()
base = np.zeros((N1,N2))
print(base.shape)
for x in range(slice_num):
    print(x)
    slice_sq = slice_array[:,:,x,0:70]
    output = Parallel(n_jobs=num_cores)(delayed(FracTool_voxel)(i,j) for i in row for j in column)
#output is j*i length list of result (2 values in each result)

    output = np.array(output) #convert output into numpy array
    output = output.astype(np.float64) #convert type object list elements to type float

    #Class_matrix = output[:,0] #first column of result output array of result is class
    Hurst_matrix = output[:,1] #second column of result output array of result is Hurst 
    Hurst_matrix = Hurst_matrix.reshape(N1,N2)
    num = x
    filename = f'Outputs/Hurst_matrix{num}.txt'
    np.savetxt(filename,Hurst_matrix,fmt='%.2f')
    base = np.dstack((base,Hurst_matrix))
    #Hurst_map = sns.heatmap(Hurst_matrix, vmin=0, vmax =1)
    #Hurst_fig = Hurst_map.get_figure()
    #Hurst_fig.suptitle('Hurst Coefficient')
    #Hurst_fig.savefig(f'Outputs/Hurst_Heatmap{num}.png')
    #plt.show()
affine = np.diag([1, 2, 3, 1])
#ni_img = nib.Nifti1Image(base, affine, n1_header)
base = base[:,:,1:slice_num+1]
ni_img = nib.nifti1.Nifti1Pair(base, None, n1_header)

nib.save(ni_img, 'Outputs/output02.nii.gz')

print(base.shape)
#Class_matrix = Class_matrix.reshape(N1,N2)
#np.savetxt('Outputs/Class_matrix.txt',Class_matrix,fmt='%.2f')
#Class_map = sns.heatmap(Class_matrix, vmin=0, vmax=3)
#Class_fig = Class_map.get_figure()
#Class_fig.suptitle('Class')
#Class_fig.savefig('Outputs/Class_Heatmap.png')
#plt.show()


35
(80, 80)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
(80, 80, 35)
